<a href="https://colab.research.google.com/github/matthewpecsok/data_engineering/blob/main/tutorials/de_tutorial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install --upgrade ipython
!pip -q install --upgrade ipython-sql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.9/808.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.19.0 which is incompatible.


In [2]:
import pandas as pd
import sqlite3

In [3]:
!wget -O movies.db https://github.com/matthewpecsok/data_engineering/blob/main/data/movies.sqlite?raw=true

con = sqlite3.connect('movies.db')

%load_ext sql
%sql sqlite:///movies.db

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

--2024-01-06 03:43:53--  https://github.com/matthewpecsok/data_engineering/blob/main/data/movies.sqlite?raw=true
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/matthewpecsok/data_engineering/raw/main/data/movies.sqlite [following]
--2024-01-06 03:43:53--  https://github.com/matthewpecsok/data_engineering/raw/main/data/movies.sqlite
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/matthewpecsok/data_engineering/main/data/movies.sqlite [following]
--2024-01-06 03:43:54--  https://raw.githubusercontent.com/matthewpecsok/data_engineering/main/data/movies.sqlite
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|18

In [4]:
%%sql SELECT name, sql FROM sqlite_master
WHERE type='table'
ORDER BY name;

,name,sql
0,directors,"CREATE TABLE `directors` (\n`name` TEXT,\n`id`..."
1,movies,CREATE TABLE `movies` (\n`id` INTEGER PRIMARY ...
2,sqlite_sequence,"CREATE TABLE sqlite_sequence(name,seq)"


In [5]:
%sql select count(1) from movies

,count(1)
0,4773


In [6]:
%sql select count(1) as director_count from directors

,director_count
0,2349


In [7]:
%sql select * from movies limit 5

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762
1,43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",285,4763
2,43599,Spectre,245000000,107,2015-10-26,880674609,Spectre,6.3,4466,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,206647,4764
3,43600,The Dark Knight Rises,250000000,112,2012-07-16,1084939099,The Dark Knight Rises,7.6,9106,Following the death of District Attorney Harve...,The Legend Ends,49026,4765
4,43601,John Carter,260000000,43,2012-03-07,284139100,John Carter,6.1,2124,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",49529,4766


In [8]:
%sql select * from directors limit 5

,name,id,gender,uid,department
0,James Cameron,4762,2,2710,Directing
1,Gore Verbinski,4763,2,1704,Directing
2,Sam Mendes,4764,2,39,Directing
3,Christopher Nolan,4765,2,525,Directing
4,Andrew Stanton,4766,2,7,Directing


In [9]:
%sql select max(budget),max(revenue) from movies

,max(budget),max(revenue)
0,380000000,2787965087


In [10]:
%sql select budget,title from movies order by budget desc limit 10

,budget,title
0,380000000,Pirates of the Caribbean: On Stranger Tides
1,300000000,Pirates of the Caribbean: At World's End
2,280000000,Avengers: Age of Ultron
3,270000000,Superman Returns
4,260000000,John Carter
5,260000000,Tangled
6,258000000,Spider-Man 3
7,255000000,The Lone Ranger
8,250000000,The Dark Knight Rises
9,250000000,Harry Potter and the Half-Blood Prince


In [11]:
%sql select min(release_date),max(release_date),max(release_date) - min(release_date) as years_of_releases from movies

,min(release_date),max(release_date),years_of_releases
0,1916-09-04,2017-02-03,101


In [24]:
%%sql select

avg(budget),
avg(revenue),
director_id

from movies

group by director_id order by (avg(budget)) desc
limit 5

,avg(budget),avg(revenue),director_id
0,2.600000e+08,5.917949e+08,4768
1,2.000000e+08,1.066970e+09,4792
2,2.000000e+08,7.435596e+08,4787
3,1.933333e+08,7.429717e+08,4770
4,1.850000e+08,5.389832e+08,4801


In [14]:
%sql select distinct(department) from directors

,department
0,Directing


In [22]:
%%sql select *

from movies m
left join directors d on m.director_id = d.id


limit 80

,id,original_title,budget,popularity,release_date,revenue,title,vote_average,vote_count,overview,tagline,uid,director_id,name,id,gender,uid,department
0,43597,Avatar,237000000,150,2009-12-10,2787965087,Avatar,7.2,11800,"In the 22nd century, a paraplegic Marine is di...",Enter the World of Pandora.,19995,4762,James Cameron,4762,2,2710,Directing
1,43598,Pirates of the Caribbean: At World's End,300000000,139,2007-05-19,961000000,Pirates of the Caribbean: At World's End,6.9,4500,"Captain Barbossa, long believed to be dead, ha...","At the end of the world, the adventure begins.",285,4763,Gore Verbinski,4763,2,1704,Directing
2,43599,Spectre,245000000,107,2015-10-26,880674609,Spectre,6.3,4466,A cryptic message from Bond’s past sends him o...,A Plan No One Escapes,206647,4764,Sam Mendes,4764,2,39,Directing
3,43600,The Dark Knight Rises,250000000,112,2012-07-16,1084939099,The Dark Knight Rises,7.6,9106,Following the death of District Attorney Harve...,The Legend Ends,49026,4765,Christopher Nolan,4765,2,525,Directing
4,43601,John Carter,260000000,43,2012-03-07,284139100,John Carter,6.1,2124,"John Carter is a war-weary, former military ca...","Lost in our world, found in another.",49529,4766,Andrew Stanton,4766,2,7,Directing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,43672,Waterworld,175000000,44,1995-07-28,264218220,Waterworld,5.9,992,In a futuristic world where the polar ice caps...,Beyond the horizon lies the secret to a new be...,9804,4814,Kevin Reynolds,4814,0,8300,Directing
76,43673,G.I. Joe: The Rise of Cobra,175000000,32,2009-08-04,302469017,G.I. Joe: The Rise of Cobra,5.6,1962,From the Egyptian desert to deep below the pol...,"When all else fails, they don't.",14869,4815,Stephen Sommers,4815,2,7775,Directing
77,43674,Inside Out,175000000,128,2015-06-09,857611174,Inside Out,8.0,6560,"Growing up can be a bumpy road, and it's no ex...",Meet the little voices inside your head.,150540,4806,Pete Docter,4806,2,12890,Directing
78,43675,The Jungle Book,175000000,94,2016-04-07,966550600,The Jungle Book,6.7,2892,After a threat from the tiger Shere Khan force...,None,278927,4808,Jon Favreau,4808,2,15277,Directing


In [36]:
%%sql select

avg(budget),
avg(revenue),
director_id,
name,
count(1) as movie_count,
max(m.title)


from movies m
left join directors d on m.director_id = d.id

group by director_id
order by avg(budget)
desc
limit 5


,avg(budget),avg(revenue),director_id,name,movie_count,max(m.title)
0,2.600000e+08,5.917949e+08,4768,Byron Howard,1,Tangled
1,2.000000e+08,1.066970e+09,4792,Lee Unkrich,1,Toy Story 3
2,2.000000e+08,7.435596e+08,4787,Dan Scanlon,1,Monsters University
3,1.933333e+08,7.429717e+08,4770,David Yates,3,The Legend of Tarzan
4,1.850000e+08,5.389832e+08,4801,Brenda Chapman,1,Brave


In [38]:
%sql select title from movies where title like '%Star Wars%'

,title
0,Star Wars: Episode III - Revenge of the Sith
1,Star Wars: Episode II - Attack of the Clones
2,Star Wars: Episode I - The Phantom Menace
3,Star Wars
4,Star Wars: Clone Wars: Volume 1
